In [1]:
from sys import path
path +=['..']

In [2]:
from mylib import Puller, PlotDisplayer, Slider, Time
import time
import numpy as np
import pandas as pd
import datetime
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, layout
from jupyter_ui_poll import ui_events
import ipywidgets
import matplotlib
import scipy
from scipy import stats
import random
import os

output_notebook()
pl = Puller(simulate=False)  #simulate=True
random.seed()

# инициализация запускать только один раз

Loading BokehJS ...

[(13, 90113195), (13, 90113196), (13, 90113197)]


In [3]:
Time.SetZeroTime()
Time.unfreez()

In [4]:
pl.ms.StopButton() # кнопки для остоновки моторчиков в нештатной ситуации

GridspecLayout(children=(Button(description='Stop All Motors', layout=Layout(grid_area='widget001'), style=But…

In [5]:
pl.ms.motorM.Set_velocity(v=20, a=20)

0

In [6]:
pl.ms.MoveToStart(zapas=10) # сдвиг моторчиков на начальные позиции для растяжки
pl.ms.motorM.MoveTo(0);
while pl.ms.IsInMotion():
    pass

In [7]:
print(pl.tg.SetZeroWeight(0.5)) # калибровка тензодатчика
print(pl.tg.SetZeroWeight(0.5))
print(pl.tg.SetZeroWeight(3))

-0.329668426411749
0.09058456761764272
-0.028214865323528064


In [8]:
# pl.sim.l0=pl.ms.Distance(useL0=False)+2

In [9]:
pl.tg.ReadValue(tau=0)

-0.002969985823529253

In [10]:
pl.SetW(20, dw=0.1, tau=1,quiet=False); # натяжка волокна

w=  15.386308557375738 , dw=  -4.613691442624262 , dx=   0.01066524835751915
w=  81.48651104489808 , dw=  61.48651104489808 , dx=   -0.14213540699162616
w=  56.19213178117379 , dw=  36.19213178117379 , dx=   -0.08366360837835422
w=  31.362060301194585 , dw=  11.362060301194585 , dx=   -0.026265127712230038
w=  21.729406280214654 , dw=  1.7294062802146541 , dx=   -0.003997785226627857
w=  20.537451969704904 , dw=  0.5374519697049038 , dx=   -0.0012424018399202418
w=  20.357272829744126 , dw=  0.3572728297441259 , dx=   -0.0008258903977435051
w=  20.163233755940215 , dw=  0.16323375594021527 , dx=   -0.0003773396138608751
w=  20.052354285195122 , dw=  0.052354285195121975 , dx=   -0.00012102488021365443
SetW finish value= 20.02463441750885


In [11]:
x0=82
pl.ms.x0=x0
# pl.sim.hotFL=True

In [12]:
while pl.ms.IsInMotion():
    pass
dp=PlotDisplayer()
# dp2=PlotDisplayer()
# dp2.CreateMaket(mainParam= 'time',obr_coff=pd.DataFrame({
#     'time': pl.times,
#     'kof': pl.trueKmas
# }))
dp.CreateMaket(mainParam='time',power=pl.data[['time','power']],
                    tension=pl.data[['time','tension','tensionWgl', 'tensionEXPgl']],
                  motors=pl.data[['time','motorR','motorL','motorM']]
                #   v_and_a=pl.data[['time','vL','vR','aL','aR']]
         )
sl=Slider()
sl.NewSl('v',min=0.1,max=9,value=5,step=0.1)
sl.NewSl('a',min=0.1,max=9,value=5,step=0.1)
sl.NewSl('dhKof',min=0, max=3,value=1,step=0.1)
sl.NewSl('window',min=50, max=10000,value=100, step=50)
sl.NewSl('molibdenH',min=-10, max=2.5,value=0.0,step=0.1)
sl.NewSl('Kp',min=0, max=10,value=0.15,step=0.01)
sl.NewSl('Ki',min=0, max=10,value=0,step=0.01)
sl.NewSl('Kd',min=-100, max=100,value=0,step=0.01)
sl.NewSl('T0',min=0, max=200,value=20,step=0.01)
sl.NewBtn('molibden','up molibden','doun molibden')
sl.NewBtn('end','end','end2','end3','vse','bolshe ne jmi','nu pravda ne nado', 'samiy umniy da?')
sl.NewTt('x')
sl.NewTt('L')
sl.NewTt('R')
sl.NewTt('dv')
sl.NewTt('T')
sl.NewTt('ki')

In [13]:
pl.MotorsControlStart()
t0 = time.time()
t = t0
k = 1
ki = 0
dp.Show()
# dp2.Show()
sl.Display()
pl.Clear()

with ui_events() as poll:
    while True:
        poll(10)
        t = time.time()  
        T = sl.Sl['T0']() * k**2
        if pl.PulMotorsControl(sl.Sl['molibdenH'](), T,
                                    upFl=sl.BtnFl['molibden'], stFl=sl.BtnFl['end'],
                                    dhKof=sl.Sl['dhKof'](), Kp=sl.Sl['Kp']()*0.001,Ki=ki,Kd=sl.Sl['Kd']()
                                 )==-1:
            break
        x=pl.data['x'].iloc[-1]
        pl.v=sl.Sl['v']()
        pl.a=sl.Sl['a']()
        ww=int(sl.Sl['window']())
        k = pl.ms.R_x(x)/pl.ms.R_x(0)
        ki = sl.Sl['Ki']()
        sl.ChangeValueTt('x',x)
        sl.ChangeValueTt('L',pl.ms.L_x(x))
        sl.ChangeValueTt('R',pl.ms.R_x(x))
        sl.ChangeValueTt('dv',pl.dv)
        sl.ChangeValueTt('T',T)
        sl.ChangeValueTt('ki',ki)
        dp.Apdate(for_all=pl.data.iloc[-ww:])
#         dp2.Apdate(for_all=pd.DataFrame({
#                     'time': pl.times,
#                     'kof': pl.trueKmas
#                 }))
        push_notebook()    
        Time.sleep(0.04)
print('ura')
# pl.Save()
# with ui_events() as poll:
#     while sl.BtnFl['end']<3:
#         poll(10)
#         pl.SetW(2,dw=1,tau=0.5)
print('vse')

starting
слишком поздний вызов PulFireMove
starting
moving vith constant speed
stoping
starting
moving vith constant speed
stoping
starting
moving vith constant speed
stoping
starting
moving vith constant speed
stoping
starting
moving vith constant speed
stoping
xMax= 38.44613234964727 L_x(xMax)= 14.700000000000001   x= -0.24817376654578993   L= -7.221194251578069 xEnd= -0.2481596386887525 lEnd= 4.721193254764884
ura
vse


In [14]:
pl.ms.End() # растановка моторчиков на нули (запускать перед отключением питания)

ending
end


In [26]:
pl.ms.Move(60, v=1, a=1, vdiff=0, da=0)

error in  motor L  -  position is bad x= -60.0 min position= 0


-1

### Для калибровки

In [2]:
pl.Clear()

dp1 = PlotDisplayer()
dp1.CreateMaket(mainParam='time', tension=pl.data[['time', 'tension']])

sl1 = Slider()
sl1.NewBtn('stop', 'stop', 'stopped')
sl1.NewTt('Tension')

dp1.Show()
sl1.Display()

with ui_events() as poll:
    while not sl1.BtnFl['stop']:
        poll(10)
        
        pl.Read(False, False, False)
        
        dp1.Apdate(for_all=pl.data.iloc[-200:])
        sl1.ChangeValueTt('Tension', pl.tg.ReadValue(tau=0))
        Time.sleep(.04)
        push_notebook()

NameError: name 'pl' is not defined

In [47]:
pl.tg.weightCoef = -0.0075585384235655265

In [50]:
pl.tg.weightCoef *= 265598 / 1000 ** 2

In [37]:
pl.tg.weightCoef *= 1000 / 9.8

In [54]:
-0.0075585384235655265 * 265598 / 1000 ** 2

-0.0020075326882221566